In [1]:
import numpy as np
import ccdredux as ccd
import imfuncs as imf
from astropy.io import fits as pf
from scipy.ndimage import filters
import shutil
from math import fabs
import glob

In [2]:
""" 
Define a function to create a temporary sky frame from
three input frames
"""

def makesky_3(infiles, medians, indices, doquad=False, xcent=127, ycent=127):
    """ 
    Makes a temporary sky file from 3 input files.
    The default behavior treats the whole image together.  However, the NIRSPEC SCAM
     appears to be read out by 4 amplifiers.  Thus, it may be better to treat the
     four quadrants independently.  If that is the case, then set doquad=True.
     NOTE: xcent and ycent represent the lower left pixel of the four pixels
     that make up the "four corners" area (where the four regions
     covered by the four different amps all touch).
     NOTE: QUADRANT SKY CREATION NOT YET IMPLEMENTED
    """
    x1 = np.array([0, 128, 0, 128])
    x2 = x1+127
    y1 = np.array([0, 0, 128, 128])
    y2 = y1+127
    tmpdat = pf.getdata(infiles[indices[0]])
    tmpshape = tmpdat.shape
    tmpsky = np.zeros((3,tmpshape[0],tmpshape[1]))
    for i in range(3):
        #if doquad:
        #    for j in range(4):
        #        tmpsky[i,y1[j]:y2[j],x1[j]:x2[j]] = ...
        tmpsky[i,:,:] = pf.getdata(infiles[indices[i]]) / medians[indices[i]]
    outsky = np.median(tmpsky,axis=0)
    del tmpdat,tmpsky
    return outsky

In [4]:
""" 
A function that makes copies of the raw data files that we can freely modify.
Unfortunately, something is screwed up with the header in the raw files,
so we are just going to copy over the data, which is non-optimal
"""
def get_raw(sciframes):
    """ 
    Inputs:
      sciframes - list or array containing the frame numbers associated with
                  the observations of the desired object
      lensroot  - rootname for output files
    """
    rawdir = '../Raw_scam/'
    rawroot = 'raw0'   
    infiles = []
    for i in sciframes:
        rawname = '%s%s%03d.fits' % (rawdir,rawroot,i)
        workname = 'work_%03d.fits' % i
        infiles.append(workname)
        shutil.copyfile(rawname,workname)
        #data = pf.getdata(rawname,ignore_missing_end=True)
        #pf.PrimaryHDU(data).writeto(workname)
        #del data
    return infiles

In [5]:
""" Get the raw data (modify for each lens-filter combination) """
sciframes = np.arange(162,171) # For J1756 J-band
lensroot = 'J1756_J'
infiles = get_raw(sciframes)
print ''
print infiles


['work_162.fits', 'work_163.fits', 'work_164.fits', 'work_165.fits', 'work_166.fits', 'work_167.fits', 'work_168.fits', 'work_169.fits', 'work_170.fits']


In [6]:
""" Make the first sky frame """
skyname = '%s_sky1.fits' % lensroot
ccd.median_combine(infiles,skyname,normalize=True)

median_combine: Inputs:
-----------------------
  bias frame: [No bias file]

median_combine: Loading files
-----------------------------
 work_162.fits
 work_163.fits
 work_164.fits
 work_165.fits
 work_166.fits
 work_167.fits
 work_168.fits
 work_169.fits
 work_170.fits

median_combine: Getting info on first file
------------------------------------------
Filename: work_162.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU     143   (256, 256)   int32   

median_combine: setting up stack for images (HDU 0)
----------------------------------------------------
Stack will have dimensions (9,256,256)
 work_162.fits
    Normalizing work_162.fits by 8801.000000
 work_163.fits
    Normalizing work_163.fits by 9056.000000
 work_164.fits
    Normalizing work_164.fits by 8690.500000
 work_165.fits
    Normalizing work_165.fits by 9608.000000
 work_166.fits
    Normalizing work_166.fits by 10212.000000
 work_167.fits
    Normalizing work_167.fits by 1133

In [8]:
""" 
Use the sky frame to make an initial bad pixel mask and then
update the sky frame itself to mask out the bad pixels.
"""
sky = imf.Image(skyname)

""" 
Do a 3-sigma clipping on the data and use the resulting clipped
mean and clipped rms to set the criterion for determining bad 
pixels
"""
sky.sigma_clip()
diff = np.fabs((sky.hdu[0].data - sky.mean_clip) / sky.rms_clip)

""" Create the bad pixel mask and write it out """
bpmask = np.fabs(diff)>5.
tmp = bpmask.astype(int)
maskname = '%s_mask_sky1.fits' % lensroot
pf.PrimaryHDU(tmp).writeto(maskname,clobber=True)
del tmp

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Save the result """
sky1v2name = '%s_sky1_v2.fits' % lensroot
pf.PrimaryHDU(skydat).writeto(sky1v2name,clobber=True)


Loading file J1756_J_sky1.fits
-----------------------------------------------
Filename: J1756_J_sky1.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   float64   
get_wcs: No valid WCS information in file header


In [9]:
""" 
Replace the bad pixels in the input files using a 2-step process:
  1. Replace each of the bad pixels with the overall median value
  2. WAIT FOR NOW ON THIS

"""
for i in infiles:
    hdu = pf.open(i,mode='update')
    data = hdu[0].data
    datmed = np.median(data)
    data[bpmask] = datmed
    hdu.flush()
    del(hdu)


In [92]:
""" Do a running sky subtraction """

""" Start by reading in all the files and calculating their median values """
alldat = np.zeros((len(infiles),bpmask.shape[0],bpmask.shape[1]))
allmed = np.zeros(len(infiles))
index_list = []
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    allmed[i] = np.median(tmp)
    if i==0:
        index_list.append([0,1,2])
    elif i==(len(infiles)-1):
        index_list.append([i-2,i-1,i])
    else:
        index_list.append([i-1,i,i+1])
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    tmpsub = tmp - allmed[i] * makesky_3(infiles,allmed,index_list[i])
    outname = 'tmpsub_%02d.fits' % sciframes[i]
    pf.PrimaryHDU(tmpsub).writeto(outname,clobber=True)
        

In [93]:
""" Do the initial sky subtraction using the updated sky frame """
sky1 = pf.getdata(sky1v2name)
skymed = np.median(sky1)
for i in infiles:
    scidat = pf.getdata(i).astype(float)
    scimed = np.median(scidat)
    scidat[bpmask] = scimed
    scimed2 = np.median(scidat)
    diff = scidat - (scimed2 / skymed) * sky1
    outfile = i.replace('work','sub1')
    pf.PrimaryHDU(diff).writeto(outfile)
    print 'Wrote sky-subtracted image (pass 1) to %s' % outfile
    del scidat,diff

Wrote sky-subtracted image (pass 1) to sub1_229.fits
Wrote sky-subtracted image (pass 1) to sub1_230.fits
Wrote sky-subtracted image (pass 1) to sub1_231.fits
Wrote sky-subtracted image (pass 1) to sub1_232.fits
Wrote sky-subtracted image (pass 1) to sub1_233.fits
Wrote sky-subtracted image (pass 1) to sub1_234.fits
Wrote sky-subtracted image (pass 1) to sub1_235.fits
Wrote sky-subtracted image (pass 1) to sub1_236.fits
Wrote sky-subtracted image (pass 1) to sub1_237.fits


In [94]:
"""
Define a function that does a quick coadd of a list of input files given
pixel offsets between them.  The quick-and-dirty aspect to this processing
is that the function will just do integer pixel shifts.
"""

def quick_coadd(filelist, offsets, outfile):
    
    """ Start by shifting the offsets to be centered on the mean offset """
    dx = offsets[:,0] - (offsets[:,0].mean())
    dy = offsets[:,1] - (offsets[:,1].mean())
    dxrange = (dx.min(),dx.max())
    dyrange = (dy.min(),dy.max())
    
    """ Make a blank image of the appropriate size """
    dat0 = pf.getdata(filelist[0])
    xsize,ysize = dat0.shape[1],dat0.shape[0]
    del dat0
    outxsize = int(xsize + fabs(dxrange[0]) + fabs(dxrange[1]))
    outysize = int(ysize + fabs(dyrange[0]) + fabs(dyrange[1]))
    outim = np.zeros((outysize,outxsize))
    
    """ Insert the data with the appropriate offsets """
    x0 = fabs(dxrange[0])
    y0 = fabs(dyrange[0])
    x0 = dxrange[1]
    y0 = dyrange[1]
    x1 = (x0 - dx).astype(int)
    x2 = x1 + int(xsize)
    y1 = (y0 - dy).astype(int)
    y2 = y1 + int(ysize)
    print 'Output file will have size: %d x %d' % (outxsize,outysize)
    print x0,y0
    print dx,dy
    print x1,x2
    print y1,y2
    for i in range(len(filelist)):
        tmp = pf.getdata(filelist[i])
        print i, tmp.shape
        try:
            outim[y1[i]:y2[i],x1[i]:x2[i]] += tmp
        except:
            print 'Failed on image %i (%s) with x1=%d,x2=%d,y1=%d,y2=%d' % (i,filelist[i],x1[i],x2[i],y1[i],y2[i])
        del tmp
    pf.PrimaryHDU(outim).writeto(outfile,clobber=True)

In [96]:
""" Do a quick-and-dirty coadd """
offsets = np.loadtxt('%s_offsets.txt' % lensroot)
#offsets = np.loadtxt('J1618_Kp_offsets.txt')
xshifts = -1. * (offsets[:,0] - offsets[0,0])
yshifts = -1. * (offsets[:,1] - offsets[0,1])
subfiles = []
for i in infiles:
    subfiles.append(i.replace('work','sub1'))
print subfiles
outfile = '%s_coadd_quick.fits' % lensroot
print outfile
quick_coadd(subfiles,offsets,outfile)
#xshifts = offsets[:,0]
#yshifts = offsets[:,1]
#ccd.coadd_intshift(subfiles,xshifts,yshifts,outfile)
print ''
print 'Wrote coadded file to %s' % outfile

['sub1_229.fits', 'sub1_230.fits', 'sub1_231.fits', 'sub1_232.fits', 'sub1_233.fits', 'sub1_234.fits', 'sub1_235.fits', 'sub1_236.fits', 'sub1_237.fits']
F2214_Kp_coadd_quick.fits
Output file will have size: 279 x 280
11.7777777778 12.1111111111
[ -0.22222222 -11.22222222  11.77777778 -11.22222222  -0.22222222
  10.77777778 -11.22222222  11.77777778  -0.22222222] [  0.11111111  10.11111111   0.11111111 -10.88888889  12.11111111
 -10.88888889   0.11111111  11.11111111 -11.88888889]
[12 23  0 23 12  1 23  0 12] [268 279 256 279 268 257 279 256 268]
[12  2 12 23  0 23 12  1 24] [268 258 268 279 256 279 268 257 280]
0 (256, 256)
1 (256, 256)
2 (256, 256)
3 (256, 256)
4 (256, 256)
5 (256, 256)
6 (256, 256)
7 (256, 256)
8 (256, 256)

Wrote coadded file to F2214_Kp_coadd_quick.fits
